# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
BASE_CHUNK  = 8192
TRAIN_CHUNK = BASE_CHUNK * 32
TOTAL_STEPS = TRAIN_CHUNK * 160
N_ENVS = 16

# Evaluation & Recording Settingsc
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1200

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [3]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [4]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None"
checkpoint_path = "runs_smw/checkpoints/Run_107.zip"

best_mean = -1e18
trained = 0
round_idx = 0

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
    trained = model.num_timesteps
    round_idx = int(trained / TRAIN_CHUNK)
    print(f"[Sucess] Loaded model from {checkpoint_path}")
    print(f"trained: {trained}, round_index: {round_idx}")
else:
    print(f"[Fail] Can't load {checkpoint_path}. Will use new model")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = 1e-4,
        verbose         = 1,
        gamma           = 0.96875,
        kl_coef         = 1,
        clip_range      = 0.125,
        ent_coef        = 0.0375,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
[Sucess] Loaded model from runs_smw/checkpoints/Run_107.zip
trained: 28311552, round_index: 108


In [5]:
# import os
# from custom_policy import CustomPPO
# from wrappers import make_base_env  # [新增] 必須引入這行來建立環境

# # ================= 設定區 =================
# # 請確保這些變數有被定義 (這裡沿用你原本的變數名稱)
# # GAME = "SuperMarioWorld-Snes"
# # STATE = "Level1" 
# # CKPT_DIR = "./"
# # RECORD_STEPS = 2000
# PSVD_DIR = "./runs_smw/preserved/"

# target_numbers = list(range(70, 128))
# # target_numbers = [124, 137, 147, 151, 179]

# # ================= 執行迴圈 =================
# for num in target_numbers:
#     model_path = os.path.join(CKPT_DIR, f"S2K_{num}.zip")
    
#     if not os.path.exists(model_path):
#         # print(f"⚠️ 找不到檔案: {model_path}，跳過。")
#         continue
    
#     # print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
#     env = None
#     try:
#         model = CustomPPO.load(model_path, device="auto")
#         env = make_base_env(game=GAME, state=STATE)
        
#         obs, info = env.reset()
#         final_score = 0
#         final_coins = 0 # [新增] 初始化金幣紀錄
        
#         for step in range(RECORD_STEPS):
#             action, _ = model.predict(obs, deterministic=True)
#             obs, reward, terminated, truncated, info = env.step(action)
            
#             # 從 info 中讀取當前數值 
#             final_score = info.get("score", final_score)
#             final_coins = info.get("coins", final_coins)
            
#             if terminated or truncated:
#                 break
        
#         # 修改後的印出格式
#         print(f"[{num}] coins: {final_coins} | score: {final_score}")
        
#     except Exception as e:
#         print(f"❌ 發生錯誤 (Model: {num}): {e}")
#     finally:
#         if env is not None:
#             env.close()

# print("\n所有測試結束。")

In [ ]:
# import os
# import glob
# from custom_policy import CustomPPO
# from eval import record_video  # 確保 eval.py 在同一目錄下
# PSVD_DIR = "./runs_smw/preserved/"
# CKPT_DIR
# # ================= 設定區 =================
# # target_numbers = list(range(38, 40))
# target_numbers = [126]

# # ================= 執行迴圈 =================
# print(f"準備測試以下 Checkpoints: {target_numbers}")

# for num in target_numbers:
#     model_path = os.path.join(PSVD_DIR, f"S2K_{num}.zip")
    
#     # 檢查檔案是否存在
#     if not os.path.exists(model_path):
#         print(f"⚠️ 找不到檔案: {model_path}，跳過。")
#         continue
    
#     print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
#     try:
#         # 1. 載入模型 (不需要 env 參數也能載入權重)
#         # 如果你有改過 CustomPPO 的參數，load 會自動讀取 zip 裡的設定
#         model = CustomPPO.load(model_path, device="auto") # device="auto" 會自動用 GPU
        
#         # 2. 錄製影片
#         prefix_name = f"test_{num}"
#         print(f"[{num}] 正在錄影 (長度 {RECORD_STEPS} steps)...")
        
#         record_video(
#             model=model,
#             game=GAME,
#             state=STATE,
#             out_dir=VIDEO_DIR,
#             video_len=RECORD_STEPS,
#             prefix=prefix_name
#         )
#         print(f"✅ 完成！影片已儲存為 {prefix_name}.mp4")
        
#     except Exception as e:
#         print(f"❌ 發生錯誤 (Model: {num}): {e}")

# print("\n所有測試結束。")

## Training Loop

In [6]:
try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000
        label = "Run"
        tagged_label = f"{label}_{int(trained/TRAIN_CHUNK)}"

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name=label)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"{tagged_label}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Record Video ---
        out_path = os.path.join(VIDEO_DIR, label)
        os.makedirs(out_path,  exist_ok=True)
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"{label}/{tagged_label}_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""


=== Round 109 | Learn 262144 steps (Total trained: 28311552) ===
Logging to ./runs_smw/tb/Run_0


/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

---------------------------------
| time/              |          |
|    fps             | 1117     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 28319744 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 904         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 28327936    |
| train/                |             |
|    approx_kl          | 0.030534374 |
|    entropy_loss       | -1.72       |
|    explained_variance | 0.951       |
|    learning_rate      | 0.0001      |
|    loss               | -0.0219     |
|    mean_step_reward   | 0.1256311   |
|    n_updates          | 4/128)      |
|    policyGradLoss     | -0.0103     |
|    value_loss         | 0.295       |
---------------------------------------
---------------------------------------
| time/                 |             |
| 

Saved checkpoint: ./runs_smw/checkpoints/Run_108.zip
[EVAL] Mean Return: 156.551, Best Return: 163.218
Saved video to ./runs_smw/videos/Run/Run_108_156.55.mp4

=== Round 110 | Learn 262144 steps (Total trained: 28573696) ===
Logging to ./runs_smw/tb/Run_0
---------------------------------
| time/              |          |
|    fps             | 1157     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 28581888 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 921         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 28590080    |
| train/                |             |
|    approx_kl          | 0.027509006 |
|    entropy_loss       | -1.76       |
|    explained_variance | 0.953       |
|    learning_rate      | 0.0001      |
|    loss               | -0.0323     |
|    mean_step_reward   | 

Saved checkpoint: ./runs_smw/checkpoints/Run_109.zip
[EVAL] Mean Return: 153.810, Best Return: 160.810
Saved video to ./runs_smw/videos/Run/Run_109_153.81.mp4

=== Round 111 | Learn 262144 steps (Total trained: 28835840) ===
Logging to ./runs_smw/tb/Run_0
---------------------------------
| time/              |          |
|    fps             | 1134     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 28844032 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 901         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 28852224    |
| train/                |             |
|    approx_kl          | 0.033149663 |
|    entropy_loss       | -1.75       |
|    explained_variance | 0.974       |
|    learning_rate      | 0.0001      |
|    loss               | -0.0376     |
|    mean_step_reward   | 

...

Saved checkpoint: ./runs_smw/checkpoints/Run_143.zip
[EVAL] Mean Return: 156.807, Best Return: 164.474
Saved video to ./runs_smw/videos/Run/Run_143_156.81.mp4

=== Round 145 | Learn 262144 steps (Total trained: 37748736) ===
Logging to ./runs_smw/tb/Run_0
---------------------------------
| time/              |          |
|    fps             | 1038     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 37756928 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 856         |
|    iterations         | 2           |
|    time_elapsed       | 19          |
|    total_timesteps    | 37765120    |
| train/                |             |
|    approx_kl          | 0.029766161 |
|    entropy_loss       | -1.73       |
|    explained_variance | 0.963       |
|    learning_rate      | 0.0001      |
|    loss               | -0.0386     |
|    mean_step_reward   | 

Saved checkpoint: ./runs_smw/checkpoints/Run_144.zip
[EVAL] Mean Return: 157.378, Best Return: 165.045
Saved video to ./runs_smw/videos/Run/Run_144_157.38.mp4

=== Round 146 | Learn 262144 steps (Total trained: 38010880) ===
Logging to ./runs_smw/tb/Run_0
---------------------------------
| time/              |          |
|    fps             | 1461     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 38019072 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 1016        |
|    iterations         | 2           |
|    time_elapsed       | 16          |
|    total_timesteps    | 38027264    |
| train/                |             |
|    approx_kl          | 0.029396765 |
|    entropy_loss       | -1.82       |
|    explained_variance | 0.966       |
|    learning_rate      | 0.0001      |
|    loss               | -0.0308     |
|    mean_step_reward   | 

Saved checkpoint: ./runs_smw/checkpoints/Run_145.zip
[EVAL] Mean Return: 154.614, Best Return: 162.614
Saved video to ./runs_smw/videos/Run/Run_145_154.61.mp4

=== Round 147 | Learn 262144 steps (Total trained: 38273024) ===
Logging to ./runs_smw/tb/Run_0
---------------------------------
| time/              |          |
|    fps             | 1053     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 38281216 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 816         |
|    iterations         | 2           |
|    time_elapsed       | 20          |
|    total_timesteps    | 38289408    |
| train/                |             |
|    approx_kl          | 0.026080651 |
|    entropy_loss       | -1.74       |
|    explained_variance | 0.971       |
|    learning_rate      | 0.0001      |
|    loss               | 0.101       |
|    mean_step_reward   | 

Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
Training interrupted manually.
T

RecursionError: maximum recursion depth exceeded

## Display Video

In [ ]:
from IPython.display import Video
import glob
# label = "Dec22A"

# list_of_files = glob.glob(os.path.join(VIDEO_DIR, label, '*.mp4')) 
# if list_of_files:
#     latest_file = max(list_of_files, key=os.path.getctime)
#     print(f"Playing: {latest_file}")
#     latest_file = "runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
#     print(f"Playing: {latest_file}")
#     display(Video(latest_file, embed=True, width=768))
# else:
#     print("No videos found yet.")
    
video = "./runs_smw/videos/test_126.mp4"
display(Video(video, embed=True, width=768))

In [ ]:
import cv2

cap = cv2.VideoCapture("runs_smw/videos/test_16.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame-by-Frame", frame)

    # 關鍵：這裡等待按鍵。按 'n' 鍵跳到下一幀，按 'q' 離開
    key = cv2.waitKey(0) 
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue

cap.release()
cv2.destroyAllWindows()

In [ ]:
"""
[070] coins: 12 | score: 3540
[071] coins: 10 | score: 2260
[072] coins: 11 | score: 2760
[073] coins:  2 | score:  690
[074] coins: 12 | score: 3450
[075] coins: 12 | score: 3515
[076] coins: 12 | score: 3545
[077] coins: 12 | score: 3545
[078] coins: 10 | score: 2460
[079] coins: 12 | score: 3515
[080] coins: 12 | score: 3580
[081] coins: 11 | score: 2750
[082] coins: 12 | score: 3545
[083] coins: 12 | score: 3565
[084] coins: 11 | score: 3475
[085] coins:  0 | score:    0
[086] coins: 12 | score: 3535
[087] coins: 12 | score: 3560
[088] coins:  9 | score: 1420
[089] coins: 11 | score: 3640
[090] coins:  1 | score:  380
[091] coins: 10 | score: 2440
[092] coins: 12 | score: 3570
[093] coins: 12 | score: 3490
[094] coins: 11 | score: 2745
[095] coins: 12 | score: 3565
[096] coins:  0 | score:    0
[097] coins: 12 | score: 3490
[098] coins: 12 | score: 3570
[099] coins:  2 | score:  560
[100] coins:  2 | score:  660
[101] coins: 12 | score: 3580
[102] coins:  9 | score: 1420
[103] coins: 12 | score: 3575
[104] coins: 12 | score: 3585
[105] coins: 12 | score: 3580
[106] coins: 12 | score: 3525
[107] coins:  2 | score:  540
[108] coins:  2 | score:  660
[109] coins: 10 | score: 2420
[110] coins:  1 | score:  140
[111] coins: 11 | score: 2680
[112] coins:  2 | score:  580
[113] coins:  2 | score:  580
[114] coins:  2 | score:  560
[115] coins: 11 | score: 2765
[116] coins:  2 | score:  560
[117] coins:  0 | score:    0
[118] coins: 12 | score: 3570
[119] coins:  1 | score:  340
[120] coins: 11 | score: 2735
[121] coins: 12 | score: 3570
[122] coins: 12 | score: 3515
[123] coins: 12 | score: 3580
[124] coins: 12 | score: 3585
[125] coins: 12 | score: 3560
[126] coins: 12 | score: 3595
[127] coins: 12 | score: 3515

所有測試結束。
在 reward 紀錄上，紀錄前10幀的 action 是甚麼，然後檢查

--Run--
[57] coins: 12 | score: 3630
[58] coins: 12 | score: 3490
[59] coins: 11 | score: 2855
[60] coins: 12 | score: 3620
[61] coins: 12 | score: 3690
[62] coins: 12 | score: 3685
[63] coins: 11 | score: 2860
[64] coins: 10 | score: 2245
[65] coins: 12 | score: 3685
[66] coins: 12 | score: 3670
[67] coins: 12 | score: 3565
[68] coins: 12 | score: 3575
[69] coins: 11 | score: 2880
[70] coins: 12 | score: 3685

[72] coins: 9 | score: 1315
[73] coins: 12 | score: 3535
[74] coins: 12 | score: 3570
[75] coins: 12 | score: 3645
[76] coins: 12 | score: 3690
[77] coins: 8 | score: 1020
[78] coins: 12 | score: 3695
[79] coins: 11 | score: 2780
[80] coins: 12 | score: 3695
[81] coins: 12 | score: 3695
[82] coins: 12 | score: 3655
[83] coins: 12 | score: 3690
[84] coins: 12 | score: 3635
[85] coins: 11 | score: 2840
[86] coins: 12 | score: 3680
[87] coins: 12 | score: 3645
[88] coins: 12 | score: 3625
[89] coins: 0 | score: 0
[90] coins: 12 | score: 3650
[91] coins: 12 | score: 3695
[92] coins: 12 | score: 3695
[93] coins: 12 | score: 3700
[94] coins: 12 | score: 3700
[95] coins: 12 | score: 3625
[96] coins: 12 | score: 3700
[97] coins: 12 | score: 3705
[98] coins: 12 | score: 3695
[99] coins: 2 | score: 640
[100] coins: 12 | score: 3705
[101] coins: 9 | score: 1400
[102] coins: 9 | score: 1420
[103] coins: 12 | score: 3705
[104] coins: 2 | score: 770
[105] coins: 12 | score: 3705
[106] coins: 12 | score: 3705
[107] coins: 1 | score: 380
[108] coins: 12 | score: 3745 *
[109] coins: 12 | score: 3660
[110] coins: 11 | score: 2915
[111] coins: 5 | score: 2810
[112] coins: 12 | score: 3680
[113] coins: 12 | score: 3540
[114] coins: 11 | score: 2860
[115] coins: 12 | score: 3740
[116] coins: 12 | score: 3730
[117] coins: 12 | score: 3725
[118] coins: 12 | score: 3680
[119] coins: 6 | score: 3650
[120] coins: 12 | score: 3745
[121] coins: 12 | score: 3770 *
[122] coins: 11 | score: 2885
[123] coins: 12 | score: 3720
[124] coins: 12 | score: 3710
[125] coins: 11 | score: 2685
[126] coins: 12 | score: 3510
[127] coins: 12 | score: 3750 *
[128] coins: 12 | score: 3730
[129] coins: 12 | score: 3635
[130] coins: 12 | score: 3730
[131] coins: 11 | score: 2745
[132] coins: 12 | score: 3720
[133] coins: 12 | score: 3760 *
[134] coins: 12 | score: 3730
[135] coins: 12 | score: 3735
[136] coins: 12 | score: 3715
"""